#### ***English to French*** 

**character-level sequence-to-sequence**

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [2]:
batch_size = 64
#epoches = 100
latent_dim = 256 # latent dim of encoding space 
num_samples = 10000 # no of samples to train

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
data_path = '/content/gdrive/My Drive/NLP labs/fra.txt'

In [5]:
### read the file 
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

lines[:3]

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tEn route !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)']

In [6]:
#### read each unique characters

input_texts = [] # all english sententces
target_texts = [] # all french sententces
input_characters = set() # all unique english char.
target_characters = set() # all unique french char.

for line in lines[: min(num_samples, len(lines)-1)]:
  input_text, target_text, _ = line.split('\t')
  #'\t' as the start seq character, '\n' as the end seq character
  target_text = '\t' + target_text + '\n' 
  input_texts.append(input_text)
  target_texts.append(target_text) 

  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [7]:
#### summary
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_len = max([len(text) for text in input_texts])
max_decoder_seq_len = max([len(text) for text in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max seq length for inputs:', max_encoder_seq_len)
print('Max seq length for inputs:', max_decoder_seq_len)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max seq length for inputs: 15
Max seq length for inputs: 59


In [8]:
### assign token indexes
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
output_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
input_token_index

In [10]:
#shape = num_pairs, max_english_sentence_length, num_english_characters
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_len, num_encoder_tokens), dtype='float32'
)
#shape (num_pairs, max_french_sentence_length, num_french_characters)
decoder_input_data  = np.zeros(
    (len(target_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32'
)
#same as decoder_input_data
decoder_target_data   = np.zeros(
    (len(target_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32'
)

In [11]:
### one hot 
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  # encoder
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  encoder_input_data[i, t+1:, input_token_index[' ']] = 1.

  # decoder
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, output_token_index[char]] = 1.
    if t>0:
      # decoder_target_data will be ahead by 1 timestep and will not include the start char
      decoder_target_data[i, t-1, output_token_index[char]] = 1.
  decoder_input_data[i, t+1:, output_token_index[' ']] = 1.
  decoder_target_data[i, t:, output_token_index[' ']] = 1.


In [12]:
encoder_input_data[0].shape

(15, 71)

In [13]:
encoder_input_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
### define an input seq and process it
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c] # we dont take encoder output


In [15]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [16]:
# model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size= batch_size,
          epochs=100,
          validation_split=0.2)


Epoch 1/100
125/125 [==============================] - 10s 19ms/step - loss: 1.1397 - accuracy: 0.7371 - val_loss: 1.0589 - val_accuracy: 0.7157
Epoch 2/100
125/125 [==============================] - 1s 12ms/step - loss: 0.8155 - accuracy: 0.7797 - val_loss: 0.8185 - val_accuracy: 0.7745
Epoch 3/100
125/125 [==============================] - 1s 12ms/step - loss: 0.6653 - accuracy: 0.8160 - val_loss: 0.6912 - val_accuracy: 0.8025
Epoch 4/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5708 - accuracy: 0.8349 - val_loss: 0.6284 - val_accuracy: 0.8197
Epoch 5/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5220 - accuracy: 0.8475 - val_loss: 0.5933 - val_accuracy: 0.8270
Epoch 6/100
125/125 [==============================] - 1s 12ms/step - loss: 0.4873 - accuracy: 0.8572 - val_loss: 0.5672 - val_accuracy: 0.8360
Epoch 7/100
125/125 [==============================] - 2s 12ms/step - loss: 0.4594 - accuracy: 0.8643 - val_loss: 0.5294 - val_accuracy

In [17]:
# define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [19]:
#reverse-lookup token index to decode seq back to somthing readable
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in output_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, output_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

for seq_index in range(30):
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Hi.
Decoded sentence: Salut.

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Hi.
Decoded sentence: Salut.

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Run!
Decoded sentence: Filez !

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Run!
Decoded sentence: Filez !

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Run!
Decoded sentence: Filez !

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Run!
Decoded sentence: Filez !

